<h2>Imports</h2>

In [15]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
from sklearn.metrics import mean_absolute_percentage_error
from catboost import CatBoostRegressor
from ydata_profiling import ProfileReport

In [16]:
csv_name = '/home/krishna/Desktop/wind_turbine/data/Kelmarsh_SCADA_2020_3086/Turbine_Data_Kelmarsh_1_20_21.csv'
csv_params = {'skiprows':9} # This part is copied from data provider check references
data_20 = pd.read_csv(csv_name,**csv_params)

In [17]:
import warnings
warnings.filterwarnings('ignore') # to avoid catboost and pd warnings

In [18]:
blade_angles = ['Blade angle (pitch position) A (°)', 'Blade angle (pitch position) B (°)', 'Blade angle (pitch position) C (°)']
data_20['blade_angle'] = data_20[blade_angles].mean(axis=1)

<h2>Data Pre-processing</h2>

In [19]:
# This part is also copied from data provider, fields look good in terms of statistics
use_columns = ['# Date and time','Power (kW)','Wind direction (°)','Nacelle position (°)','blade_angle','Rear bearing temperature (°C)',
            'Rotor speed (RPM)','Generator RPM (RPM)','Nacelle ambient temperature (°C)',
            'Front bearing temperature (°C)','Tower Acceleration X (mm/ss)','Wind speed (m/s)','Tower Acceleration y (mm/ss)','Metal particle count counter',]
input = '# Date and time'
output = ['Power (kW)','Wind speed (m/s)','Wind speed (m/s)']

In [20]:
data_20.drop(data_20.filter(like='Maximum').columns, axis=1, inplace=True)
data_20.drop(data_20.filter(like='Minimum').columns, axis=1, inplace=True)
data_20.drop(data_20.filter(like='1').columns, axis=1, inplace=True)
data_20.drop(data_20.filter(like='2').columns, axis=1, inplace=True)
data_20.drop(data_20.filter(like='Available Capacity').columns, axis=1, inplace=True)
data_20.drop(data_20.filter(like='Reactive Energy Export').columns, axis=1, inplace=True)
data_20.drop(data_20.filter(like='Avail.').columns, axis=1, inplace=True)
data_20.drop(data_20.filter(like='Potential power').columns, axis=1, inplace=True)
data_20.drop(data_20.filter(like='Blade angle').columns, axis=1, inplace=True) 


In [21]:
notusecols=[]

# drop all the columns except those in usecolumns
for col in data_20.columns:
    if col not in use_columns:
        notusecols.append(col)

data_20 = data_20.drop(columns=[col for col in data_20.columns if col not in use_columns])

print(data_20.columns)


Index(['# Date and time', 'Wind speed (m/s)', 'Wind direction (°)',
       'Nacelle position (°)', 'Power (kW)', 'Front bearing temperature (°C)',
       'Rear bearing temperature (°C)', 'Nacelle ambient temperature (°C)',
       'Rotor speed (RPM)', 'Generator RPM (RPM)',
       'Metal particle count counter', 'Tower Acceleration X (mm/ss)',
       'Tower Acceleration y (mm/ss)', 'blade_angle'],
      dtype='object')


In [22]:
data_20_20 = data_20[:2000]
profile = ProfileReport(data_20[use_columns])
profile.to_file("output.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [23]:

data_20 = data_20[(data_20['Wind speed (m/s)'] <= 16) & (data_20['Wind speed (m/s)'] >= 3)]

# Reset the index after dropping rows
data_20 = data_20.reset_index(drop=True)
data_20 = data_20.reset_index()



In [24]:
!pip install lightgbm

In [25]:
from Wind.dataset import Dataset
from Wind.utils import score_function
from Wind.model import *

In [12]:
import warnings
warnings.filterwarnings('ignore') # to avoid catboost and pd warnings
!rm output.html

In [13]:
data_20.dropna(inplace=True)

In [14]:

from darts import TimeSeries
from darts.utils.timeseries_generation import linear_timeseries

frequency = '10min'

# Assuming you have a pandas DataFrame with a column named 'timestamp' and a column named 'power'
# and you want to split the data into 80% training and 20% testing

# Convert your data to a Darts TimeSeries object
series = TimeSeries.from_dataframe(data_20, '# Date and time', 'Power (kW)', freq=frequency)

# Split the time series into training and testing sets
train_size = int(len(series) * 0.8)  # 80% of the data for training
train, test = series[:train_size], series[train_size:]

# Print the lengths of the train and test sets
print("Train length:", len(train))
print("Test length:", len(test))



AttributeError: time_col='# Date and time' is not present.


AttributeError: time_col='# Date and time' is not present.

In [ ]:
# This part is also copied from data provider, fields look good in terms of statistics
use_columns = ['Wind speed (m/s)','Wind speed, Standard deviation (m/s)','Wind direction (°)','Nacelle position (°)','Energy Export (kWh)','Reactive power (kvar)','Rotor speed (RPM)','Generator RPM (RPM)','Nacelle ambient temperature (°C)',
            'Blade angle (pitch position) A (°)','Blade angle (pitch position) B (°)','Blade angle (pitch position) C (°)',
            'Lost Production to Curtailment (Total) (kWh)','Lost Production to Downtime (kWh)',
            'Front bearing temperature (°C)','Rear bearing temperature (°C)']

In [ ]:
from sklearn.linear_model import LinearRegression

# Assuming you have already split the data into train and test sets

# Create a LinearRegression object
model = LinearRegression()

# Fit the model to the training set
model.fit(train.pd_dataframe().values[:-1], train.pd_dataframe().values[1:])

# Perform a one-step ahead prediction using the fitted model on the test set
prediction = model.predict(test.pd_dataframe().values[:-1])


ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values